In [440]:
from matplotlib.pyplot import draw, fill
import pyrender
import trimesh
from trimesh import remesh
import pandas as pd
import numpy as np
from trimesh import Trimesh
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import math
import os
import matplotlib.mlab as mlab
import seaborn as sns
import skeletor as sk
from view_data import view_mesh
import copy
from sklearn.preprocessing import MinMaxScaler
from project_statistics import get_outliers
from create_dataset import load_dataset,dir_to_sorted_file_list
from trimesh.repair import fill_holes
from trimesh.points import PointCloud
import ast
import random
from scipy.stats import wasserstein_distance
from random import choice

In [530]:
with open('normalized_final_features_df.pkl', 'rb') as f:
    normalized_data = pickle.load(f)

In [397]:
with open('example_221.pkl', 'rb') as f:
    query = pickle.load(f)

In [398]:
query

,id,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
0,221,-0.588866,-0.50444,-0.00564,-0.784274,1.0,-0.403296,-0.026324,"[0.12004230565838181, 0.026969857218402962, 0....","[0.2923697357597159, 0.18890070423948155, 0.14...","[0.13017395317108404, 0.06781565377901727, 0.0...","[0.10312794687104994, 0.17491623362292089, 0.1...","[0.2077807427619831, 0.21638811420316606, 0.14...","[0.05692100880800947, 0.22688148311630987, 0.2..."


In [531]:
normalized_data

,id,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
0,0,-0.583110,-0.039329,-0.116012,0.230250,1.0,-0.109650,-0.026311,"[0.0015568240788790867, 0.011416709911779968, ...","[0.23664198719878007, 0.22873653163185456, 0.1...","[0.04733241036136359, 0.12355785629727724, 0.1...","[0.09370171953690884, 0.19420534119865968, 0.2...","[0.20435803286582796, 0.15628323200706273, 0.1...","[0.13328919119565, 0.28198800136439334, 0.2750..."
1,1,-0.640930,-0.415322,-0.118042,0.272779,1.0,0.097077,-0.026489,"[0.04924623115577889, 0.054773869346733665, 0....","[0.24772371746019012, 0.22417873330391883, 0.1...","[0.12446351931330472, 0.25626729774176726, 0.1...","[0.151618466968834, 0.300790180899282, 0.27189...","[0.3746741005174281, 0.28757370342144317, 0.16...","[0.07649111547872127, 0.2777064698568048, 0.32..."
2,2,-0.603237,-0.095673,-0.118419,0.280905,1.0,-0.043228,-0.026263,"[0.015625, 0.03780241935483871, 0.137096774193...","[0.25301446542142325, 0.23644242922777522, 0.1...","[0.15300042132295408, 0.21009971309913128, 0.1...","[0.14629938005336757, 0.26599522500652045, 0.2...","[0.29506650883775054, 0.24288265152579097, 0.1...","[0.11953534097064783, 0.3514635956904681, 0.29..."
3,3,-0.520618,0.392970,-0.119893,0.313312,1.0,-0.468747,-0.026398,"[0.011866235167206042, 0.02481121898597627, 0....","[0.20333848283610537, 0.23802740605501274, 0.1...","[0.05069919547378769, 0.09237004193166543, 0.0...","[0.061753907268824106, 0.1599622815641113, 0.1...","[0.1285636899865578, 0.1484260578215597, 0.138...","[0.08151596011476035, 0.2712316674357482, 0.31..."
4,4,-0.724641,-0.475575,-0.095315,-0.114249,1.0,-0.306213,-0.026249,"[0.06380952380952382, 0.10952380952380952, 0.2...","[0.2399117529081428, 0.2335138387484958, 0.155...","[0.17721620537505015, 0.18864821500200563, 0.2...","[0.1078018451664661, 0.2600882470918572, 0.267...","[0.2778780585639791, 0.25130365022061774, 0.16...","[0.07882069795427196, 0.2884476534296029, 0.31..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,1809,-0.365028,-0.011320,-0.113034,0.171236,1.0,-0.081913,-0.026487,"[0.08372530573847602, 0.009877704609595485, 0....","[0.1711191335740072, 0.23331327717609307, 0.20...","[0.10609707180104291, 0.027476935419173686, 0....","[0.10146409947853992, 0.1478740473325311, 0.25...","[0.1829121540312876, 0.11217408744484557, 0.09...","[0.07059767348576013, 0.20198555956678702, 0.2..."
1810,1810,-0.089583,0.813581,-0.138504,0.844492,1.0,-0.125955,-0.026451,"[0.0903030303030303, 0.15575757575757576, 0.38...","[0.16446496687412165, 0.24053402931138326, 0.2...","[0.18598674964866493, 0.2187111021883156, 0.21...","[0.07153182091949407, 0.1783778357759486, 0.24...","[0.13278458140935556, 0.1505320216823931, 0.14...","[0.08239309375627384, 0.27639028307568764, 0.3..."
1811,1811,-0.509943,-0.716180,-0.046532,-0.572259,1.0,-0.403390,-0.026294,"[0.008452192287374538, 0.0306391970417327, 0.0...","[0.2840779153042187, 0.1652189612630143, 0.140...","[0.1258199763285121, 0.4619952255812554, 0.094...","[0.25559188750025075, 0.2389616642259624, 0.12...","[0.4473911211859817, 0.21958314108607996, 0.13...","[0.10658187726935345, 0.2844189452145479, 0.28..."
1812,1812,0.413064,1.070660,-0.142995,1.019764,1.0,-0.306231,-0.026260,"[0.1552892131318395, 0.26003126628452317, 0.28...","[0.23453768531707023, 0.2171845848295785, 0.16...","[0.06275202118482556, 0.09860172126707725, 0.1...","[0.15445262503259977, 0.1181415130298714, 0.18...","[0.29749032038036394, 0.104580014845427, 0.078...","[0.2087588019339178, 0.29097036933015025, 0.20..."


In [556]:
num_samples = 5
descriptor_sample_data = {}

for index, row in normalized_data.iterrows():
    print(index, end='\r')
    while(True):
        samples = random.sample(range(0, 1813), num_samples)
        if (index in samples or len(np.unique(samples))!=len(samples)):
            continue
        else:
            break

    for s in range(len(samples)):
        s_id = samples[s]
        sample_decriptors = normalized_data.loc[s_id][8:]
        
        for feature, column in enumerate(normalized_data.columns[8:]):
            sample_descriptor_feature = sample_decriptors[feature]
            original_descriptor_feature = row[column]
            d = wasserstein_distance(original_descriptor_feature, sample_descriptor_feature)
            
            if column not in descriptor_sample_data.keys():
                descriptor_sample_data[column] = []
            else:
                descriptor_sample_data[column].append(d)

In [561]:
len(descriptor_sample_data['curvature'])

9069

In [557]:
sampled_mean_std = []
for key in list(descriptor_sample_data.keys()):
    sampled_emd = descriptor_sample_data[key]
    sampled_mean = np.mean(sampled_emd)
    sampled_std = np.std(sampled_emd)
    sampled_mean_std.append((sampled_mean,sampled_std))

In [555]:
sampled_mean_std

[(0.059429441878822084, 0.03226593254867776),
 (0.027214808621970836, 0.016921542233045206),
 (0.051924359479794006, 0.030803306082639266),
 (0.04112516544977415, 0.027038918195671342),
 (0.062236344692158434, 0.036933804258945685),
 (0.03552519593983209, 0.026008049608010377)]

In [558]:
sampled_mean_std

[(0.0597939350892226, 0.032585130661088355),
 (0.0273437109278185, 0.017008402417438066),
 (0.052098248458764325, 0.030820432174388453),
 (0.041069724341105014, 0.026820812107617387),
 (0.06203270048335279, 0.03716688454254701),
 (0.03585467046597278, 0.02630622529945391)]

In [549]:
def standardize(data, idx):
    #sampled_mean_std = np.load('sampled_mean_std.npy')
    #sampled_mean_std = sampled_mean_std
    data_mean = sampled_mean_std[idx][0]  
    data_std = sampled_mean_std[idx][1]  
    normalized_descriptor_data = (data - data_mean)/data_std
    return normalized_descriptor_data

In [559]:
normalized_data

,id,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
0,0,-0.583110,-0.039329,-0.116012,0.230250,1.0,-0.109650,-0.026311,"[0.0015568240788790867, 0.011416709911779968, ...","[0.23664198719878007, 0.22873653163185456, 0.1...","[0.04733241036136359, 0.12355785629727724, 0.1...","[0.09370171953690884, 0.19420534119865968, 0.2...","[0.20435803286582796, 0.15628323200706273, 0.1...","[0.13328919119565, 0.28198800136439334, 0.2750..."
1,1,-0.640930,-0.415322,-0.118042,0.272779,1.0,0.097077,-0.026489,"[0.04924623115577889, 0.054773869346733665, 0....","[0.24772371746019012, 0.22417873330391883, 0.1...","[0.12446351931330472, 0.25626729774176726, 0.1...","[0.151618466968834, 0.300790180899282, 0.27189...","[0.3746741005174281, 0.28757370342144317, 0.16...","[0.07649111547872127, 0.2777064698568048, 0.32..."
2,2,-0.603237,-0.095673,-0.118419,0.280905,1.0,-0.043228,-0.026263,"[0.015625, 0.03780241935483871, 0.137096774193...","[0.25301446542142325, 0.23644242922777522, 0.1...","[0.15300042132295408, 0.21009971309913128, 0.1...","[0.14629938005336757, 0.26599522500652045, 0.2...","[0.29506650883775054, 0.24288265152579097, 0.1...","[0.11953534097064783, 0.3514635956904681, 0.29..."
3,3,-0.520618,0.392970,-0.119893,0.313312,1.0,-0.468747,-0.026398,"[0.011866235167206042, 0.02481121898597627, 0....","[0.20333848283610537, 0.23802740605501274, 0.1...","[0.05069919547378769, 0.09237004193166543, 0.0...","[0.061753907268824106, 0.1599622815641113, 0.1...","[0.1285636899865578, 0.1484260578215597, 0.138...","[0.08151596011476035, 0.2712316674357482, 0.31..."
4,4,-0.724641,-0.475575,-0.095315,-0.114249,1.0,-0.306213,-0.026249,"[0.06380952380952382, 0.10952380952380952, 0.2...","[0.2399117529081428, 0.2335138387484958, 0.155...","[0.17721620537505015, 0.18864821500200563, 0.2...","[0.1078018451664661, 0.2600882470918572, 0.267...","[0.2778780585639791, 0.25130365022061774, 0.16...","[0.07882069795427196, 0.2884476534296029, 0.31..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,1809,-0.365028,-0.011320,-0.113034,0.171236,1.0,-0.081913,-0.026487,"[0.08372530573847602, 0.009877704609595485, 0....","[0.1711191335740072, 0.23331327717609307, 0.20...","[0.10609707180104291, 0.027476935419173686, 0....","[0.10146409947853992, 0.1478740473325311, 0.25...","[0.1829121540312876, 0.11217408744484557, 0.09...","[0.07059767348576013, 0.20198555956678702, 0.2..."
1810,1810,-0.089583,0.813581,-0.138504,0.844492,1.0,-0.125955,-0.026451,"[0.0903030303030303, 0.15575757575757576, 0.38...","[0.16446496687412165, 0.24053402931138326, 0.2...","[0.18598674964866493, 0.2187111021883156, 0.21...","[0.07153182091949407, 0.1783778357759486, 0.24...","[0.13278458140935556, 0.1505320216823931, 0.14...","[0.08239309375627384, 0.27639028307568764, 0.3..."
1811,1811,-0.509943,-0.716180,-0.046532,-0.572259,1.0,-0.403390,-0.026294,"[0.008452192287374538, 0.0306391970417327, 0.0...","[0.2840779153042187, 0.1652189612630143, 0.140...","[0.1258199763285121, 0.4619952255812554, 0.094...","[0.25559188750025075, 0.2389616642259624, 0.12...","[0.4473911211859817, 0.21958314108607996, 0.13...","[0.10658187726935345, 0.2844189452145479, 0.28..."
1812,1812,0.413064,1.070660,-0.142995,1.019764,1.0,-0.306231,-0.026260,"[0.1552892131318395, 0.26003126628452317, 0.28...","[0.23453768531707023, 0.2171845848295785, 0.16...","[0.06275202118482556, 0.09860172126707725, 0.1...","[0.15445262503259977, 0.1181415130298714, 0.18...","[0.29749032038036394, 0.104580014845427, 0.078...","[0.2087588019339178, 0.29097036933015025, 0.20..."


In [578]:
comaprsion_df = normalized_data.copy()
for column in query.columns[1:8]:
    comaprsion_df[column] = normalized_data[column] - query[column][0]

for idx, column in enumerate(normalized_data.columns[8:]):
    comaprsion_df[column] = [standardize(wasserstein_distance(row, query[column][0]), idx) for row in normalized_data[column]]

In [579]:
comaprsion_df

,id,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
0,0,0.005756,0.465111,-0.110372,1.014524,0.0,0.293646,0.000013,2.236909,-0.315027,-0.607301,-0.189868,-1.140985,-0.599321
1,1,-0.052064,0.089118,-0.112402,1.057053,0.0,0.500373,-0.000165,1.822141,-0.557372,-0.781702,0.740207,-0.065640,-0.493730
2,2,-0.014371,0.408767,-0.112780,1.065179,0.0,0.360068,0.000061,1.923061,-0.521833,-0.725733,0.466318,-0.903698,-0.257896
3,3,0.068248,0.897410,-0.114253,1.097586,0.0,-0.065451,-0.000075,0.513960,-0.160358,-1.232713,-0.486347,-0.978207,-0.547139
4,4,-0.135775,0.028865,-0.089675,0.670024,0.0,0.097083,0.000075,2.683910,-0.464479,-0.182416,0.383486,-0.921397,-0.461641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,1809,0.223838,0.493121,-0.107395,0.955510,0.0,0.321383,-0.000163,0.411564,0.225072,-0.195249,-0.048060,-0.904675,0.011726
1810,1810,0.499283,1.318021,-0.132864,1.628766,0.0,0.277341,-0.000128,1.269699,0.019329,0.216623,0.464132,-0.750312,-0.528403
1811,1811,0.078922,-0.211739,-0.040892,0.212015,0.0,-0.000094,0.000030,2.082512,-0.772825,0.482675,0.112727,-0.035821,-0.537937
1812,1812,1.001929,1.575100,-0.137356,1.804038,0.0,0.097065,0.000064,2.039981,-0.403741,-0.660664,-0.239786,-0.537151,-0.750804


In [581]:
comaprsion_df = comaprsion_df.drop('id', axis=1)

In [588]:
comaprsion_df

,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
0,0.005756,0.465111,-0.110372,1.014524,0.0,0.293646,0.000013,2.236909,-0.315027,-0.607301,-0.189868,-1.140985,-0.599321
1,-0.052064,0.089118,-0.112402,1.057053,0.0,0.500373,-0.000165,1.822141,-0.557372,-0.781702,0.740207,-0.065640,-0.493730
2,-0.014371,0.408767,-0.112780,1.065179,0.0,0.360068,0.000061,1.923061,-0.521833,-0.725733,0.466318,-0.903698,-0.257896
3,0.068248,0.897410,-0.114253,1.097586,0.0,-0.065451,-0.000075,0.513960,-0.160358,-1.232713,-0.486347,-0.978207,-0.547139
4,-0.135775,0.028865,-0.089675,0.670024,0.0,0.097083,0.000075,2.683910,-0.464479,-0.182416,0.383486,-0.921397,-0.461641
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,0.223838,0.493121,-0.107395,0.955510,0.0,0.321383,-0.000163,0.411564,0.225072,-0.195249,-0.048060,-0.904675,0.011726
1810,0.499283,1.318021,-0.132864,1.628766,0.0,0.277341,-0.000128,1.269699,0.019329,0.216623,0.464132,-0.750312,-0.528403
1811,0.078922,-0.211739,-0.040892,0.212015,0.0,-0.000094,0.000030,2.082512,-0.772825,0.482675,0.112727,-0.035821,-0.537937
1812,1.001929,1.575100,-0.137356,1.804038,0.0,0.097065,0.000064,2.039981,-0.403741,-0.660664,-0.239786,-0.537151,-0.750804


In [592]:
comparsion_values = comaprsion_df.sum(axis = 1, skipna = True)
comparsion_values.sort_values(ascending=True)

221     -9.400475
1236    -6.169224
1155    -6.026556
160     -5.943403
223     -5.482214
          ...    
560     26.621329
84      31.253593
704     33.479203
1692    33.989434
1707    39.989479
Length: 1814, dtype: float64

In [564]:
comaprsion_df.describe()

,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature,A3,D1,D2,D3,D4
count,1814.000000,1814.000000,1814.000000,1814.000000,1814.0,1814.000000,1813.000000,1814.000000,1814.000000,1814.000000,1814.000000,1814.000000,1814.000000
mean,0.588866,0.504440,0.005640,0.784274,0.0,0.403296,0.026324,1.215862,-0.187769,0.138782,0.202479,-0.132144,-0.280317
std,1.000276,1.000276,1.000276,1.000276,0.0,1.000276,1.000276,1.121622,0.585559,1.141851,1.027968,1.069507,0.761823
min,-0.350812,-0.451257,-0.191027,-0.822044,0.0,-1.478186,-0.000168,-1.841863,-1.509682,-1.636835,-1.503336,-1.649351,-1.289327
25%,-0.006678,-0.202018,-0.121928,0.152121,0.0,-0.188722,-0.000162,0.521575,-0.646236,-0.734160,-0.484667,-0.961155,-0.679493
50%,0.295789,0.132908,-0.093812,0.728173,0.0,0.242485,-0.000137,1.459475,-0.169690,-0.102883,-0.042313,-0.434048,-0.447976
75%,0.837997,0.921425,-0.031136,1.286046,0.0,0.792199,-0.000042,2.100986,0.232058,0.788649,0.594900,0.443569,-0.108679
max,19.325962,7.189479,38.288247,17.075289,0.0,13.815189,42.387499,3.074064,4.046008,4.631338,5.415124,3.514023,5.329263


In [566]:
normalized_data.describe()

,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity
count,1.814000e+03,1.814000e+03,1.814000e+03,1.814000e+03,1814.0,1.814000e+03,1.813000e+03
mean,-6.267191e-17,-5.483792e-17,-1.566798e-17,-1.175098e-16,1.0,3.133595e-17,-1.567662e-17
std,1.000276e+00,1.000276e+00,1.000276e+00,1.000276e+00,0.0,1.000276e+00,1.000276e+00
min,-9.396772e-01,-9.556979e-01,-1.966668e-01,-1.606318e+00,1.0,-1.881482e+00,-2.649182e-02
25%,-5.955436e-01,-7.064586e-01,-1.275672e-01,-6.321529e-01,1.0,-5.920179e-01,-2.648564e-02
50%,-2.930762e-01,-3.715324e-01,-9.945174e-02,-5.610093e-02,1.0,-1.608111e-01,-2.646072e-02
75%,2.491312e-01,4.169846e-01,-3.677571e-02,5.017720e-01,1.0,3.889033e-01,-2.636570e-02
max,1.873710e+01,6.685038e+00,3.828261e+01,1.629102e+01,1.0,1.341189e+01,4.236118e+01
